In [68]:
import argparse
import numpy as np

RATING_FILE_NAME = dict({'movie': 'ratings.dat', 'book': 'BX-Book-Ratings.csv', 'news': 'ratings.txt'})
SEP = dict({'movie': '::', 'book': ';', 'news': '\t'})
THRESHOLD = dict({'movie': 4, 'book': 0, 'news': 0})

args = argparse.Namespace()
args.dataset="naver"
DATASET = args.dataset
entity_id2index = dict()
relation_id2index = dict()
item_index_old2new = dict()

In [7]:
def read_item_index_to_entity_id_file():
    file = './data/' + DATASET + '/item_index2entity_id_rehashed.txt'
    print('reading item index to entity id file: ' + file + ' ...')
    i = 0
    for line in open(file, encoding='utf-8').readlines():
        item_index = line.strip().split('\t')[0]
        satori_id = line.strip().split('\t')[1]
        item_index_old2new[item_index] = i
        entity_id2index[satori_id] = i
        i += 1
read_item_index_to_entity_id_file()

reading item index to entity id file: ./data/movie/item_index2entity_id_rehashed.txt ...


reading rating file ...


In [62]:
def negative_samplig(user_set, user_pos_ratings):
    neg_items = dict()
    for user_idx in user_set:
        neg_items[user_idx] = set()
    for user_idx in user_set:
        
        while True:
            neg_item = np.random.randint(low=0, high=len(item_set), size=1)[0]
            if neg_item in user_pos_ratings[user_idx]:
                continue
            # print(user_idx, neg_item)
            neg_items[user_idx].add(neg_item)
            if len(neg_items[user_idx]) == len(user_pos_ratings[user_idx]):
                break
    return neg_items



In [63]:


file = './data/' + DATASET + '/' + RATING_FILE_NAME[DATASET]

print('reading rating file ...')
item_set = set(item_index_old2new.values())
user_pos_ratings = dict()
user_neg_ratings = dict()

for line in open(file, encoding='utf-8').readlines()[1:]:
    array = line.strip().split(SEP[DATASET])

    # remove prefix and suffix quotation marks for BX dataset
    if DATASET == 'book':
        array = list(map(lambda x: x[1:-1], array))

    item_index_old = array[1]
    if item_index_old not in item_index_old2new:  # the item is not in the final item set
        continue
    item_index = item_index_old2new[item_index_old]

    user_index_old = int(array[0])

    rating = float(array[2])
    
    
    if user_index_old not in user_pos_ratings:
        user_pos_ratings[user_index_old] = set()
    user_pos_ratings[user_index_old].add(item_index)

user_set = set(user_pos_ratings.keys())    
user_neg_ratings = negative_samplig(user_set, user_pos_ratings)    

    


reading rating file ...


In [66]:

print('converting rating file ...')
writer = open('./data/' + DATASET + '/ratings_final_temp.txt', 'w', encoding='utf-8')
user_cnt = 0
user_index_old2new = dict()
for user_index_old, pos_item_set in user_pos_ratings.items():
    if user_index_old not in user_index_old2new:
        user_index_old2new[user_index_old] = user_cnt
        user_cnt += 1
    user_index = user_index_old2new[user_index_old]

    for item in pos_item_set:
        writer.write('%d\t%d\t1\n' % (user_index, item))
        
    unwatched_set = item_set - pos_item_set
    if user_index_old in user_neg_ratings:
        unwatched_set -= user_neg_ratings[user_index_old]
    for item in np.random.choice(list(unwatched_set), size=len(pos_item_set), replace=False):
        writer.write('%d\t%d\t0\n' % (user_index, item))
writer.close()
print('number of users: %d' % user_cnt)
print('number of items: %d' % len(item_set))


converting rating file ...


ValueError: Cannot take a larger sample than population when 'replace=False'